In [2]:
!pip install pykalman


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pykalman]


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Import statsmodels for ARIMA
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Import pykalman for Kalman Filter
from pykalman import KalmanFilter

# Import utilities
import warnings
warnings.filterwarnings('ignore')

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Define the column names based on the readme.txt
col_names = ['unit_number', 'time_in_cycles', 'setting_1', 'setting_2', 'setting_3']
col_names.extend([f'sensor_{i}' for i in range(1, 22)])

# Load the training data
df_train = pd.read_csv('/CMAPSSData/train_FD001.txt', sep=' ', header=None, names=col_names)

# Drop the last two columns which are empty (NaNs)
df_train = df_train.drop(columns=[26, 27])

# Display the first few rows
print("Data loaded successfully:")
display(df_train.head())

# Display data info
print("\nData Info:")
df_train.info()

In [ ]:
# Calculate standard deviation for each feature
data_std = df_train.std()
print("Standard Deviations:")
print(data_std)

# Identify columns with zero (or very low) variance
# These columns provide no information for the FD001 dataset
cols_to_drop = data_std[data_std < 1e-5].index.tolist()
print(f"\nConstant columns to drop: {cols_to_drop}")

# Drop these columns from the dataframe
df_train_processed = df_train.drop(columns=cols_to_drop)

# Display the shape of the new dataframe
print(f"\nOriginal shape: {df_train.shape}")
print(f"Processed shape: {df_train_processed.shape}")

In [ ]:
# Get the list of remaining sensor columns
sensor_cols = [col for col in df_train_processed.columns if 'sensor' in col]
print(f"Remaining sensors to scale: {sensor_cols}")

# Initialize the MinMax scaler
scaler = MinMaxScaler()

# Create a copy to hold the scaled data
df_train_scaled = df_train_processed.copy()

# Fit and transform the sensor data
df_train_scaled[sensor_cols] = scaler.fit_transform(df_train_scaled[sensor_cols])

print("\nScaled Data Head:")
display(df_train_scaled.head())

In [ ]:
# Select all data for unit 1
engine_1_data = df_train_scaled[df_train_scaled['unit_number'] == 1].copy()

# Select sensor_7 as our time series
ts = engine_1_data['sensor_7'].reset_index(drop=True)

# Plot the chosen sensor data
plt.figure(figsize=(12, 5))
plt.plot(ts, label='Sensor 7 - Unit 1')
plt.title('Full Time Series (Sensor 7, Unit 1)')
plt.xlabel('Time in Cycles')
plt.ylabel('Normalized Value')
plt.legend()
plt.show()

In [ ]:
# Define the split point
split_point = 50 

# Create training and testing sets
train_data = ts.iloc[:split_point]
test_data = ts.iloc[split_point:]

# Plot the split
plt.figure(figsize=(12, 5))
plt.plot(train_data, label='Train (Healthy)')
plt.plot(test_data, label='Test (Degrading)')
plt.title('Train/Test Split')
plt.xlabel('Time in Cycles')
plt.axvline(split_point, color='red', linestyle='--', label='Split')
plt.legend()
plt.show()

In [ ]:
# --- 1. Find 'd' (Order of Differencing) ---
print("--- Finding 'd' (Differencing) ---")
adf_result = adfuller(train_data)
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')

# If p-value is > 0.05, we need to difference
if adf_result[1] > 0.05:
    print("Data is non-stationary. Differencing once (d=1).")
    d = 1
    # Check p-value after differencing
    adf_result_diff = adfuller(train_data.diff().dropna())
    print(f'p-value after 1st diff: {adf_result_diff[1]}')
else:
    print("Data is stationary (d=0).")
    d = 0

# --- 2. Find 'p' and 'q' ---
print("\n--- Finding 'p' and 'q' ---")
# Plot ACF and PACF on the (now stationary) data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(train_data.diff().dropna(), lags=20, ax=ax1)
plot_pacf(train_data.diff().dropna(), lags=20, ax=ax2, method='ywm')
plt.show()

print("""
Based on the plots (example interpretation):
- PACF (top): Cuts off sharply after lag 1. Suggests p=1.
- ACF (bottom): Tails off.
Let's choose p=1, d=1, q=1 as a robust starting point.
""")

In [ ]:
print("Starting ARIMA rolling forecast...")

history = list(train_data)
predictions = []
anomaly_scores_arima = []

for t in range(len(test_data)):
    # Create and fit the ARIMA model
    model = ARIMA(history, order=(1, 1, 1))
    model_fit = model.fit()
    
    # Forecast the next step
    yhat = model_fit.forecast()[0]
    predictions.append(yhat)
    
    # Get the true observation
    obs = test_data.iloc[t]
    
    # Calculate the anomaly score (Absolute Error)
    error = abs(obs - yhat)
    anomaly_scores_arima.append(error)
    
    # Add the new observation to history
    history.append(obs)

print("ARIMA forecast complete.")

# Store results in a dataframe
arima_results = pd.DataFrame({
    'actual': test_data.values,
    'predicted': predictions,
    'anomaly_score': anomaly_scores_arima
}, index=test_data.index)

display(arima_results.head())

In [ ]:
# Plot 1: Actual vs. Predicted
plt.figure(figsize=(12, 5))
plt.plot(arima_results['actual'], label='Actual Value')
plt.plot(arima_results['predicted'], label='ARIMA Prediction', linestyle='--', alpha=0.7)
plt.title('ARIMA: Actual vs. Predicted')
plt.legend()
plt.show()

# Plot 2: Anomaly Scores
plt.figure(figsize=(12, 5))
plt.plot(arima_results['anomaly_score'], label='ARIMA Anomaly Score', color='orange')
plt.title('ARIMA Anomaly Scores (Absolute Error)')
plt.xlabel('Time in Cycles')
plt.ylabel('Anomaly Score')
plt.legend()
plt.show()

In [ ]:
# We use the full time series `ts` for smoothing
data_full = ts.values

# Configure the Kalman Filter for a simple state model
# (e.g., value = last_value + noise)
kf = KalmanFilter(
    transition_matrices = [1],         # State evolves by 1*last_state (A)
    observation_matrices = [1],        # Observation is 1*current_state (H)
    initial_state_mean = data_full[0], # Initial guess
    initial_state_covariance = 1,      # Initial uncertainty
    observation_covariance = 1,        # Measurement noise (R) - TUNE THIS
    transition_covariance = 0.01       # Process noise (Q) - TUNE THIS
)

# Run the Kalman Filter smoother over all data
(smoothed_state_means, smoothed_state_covariances) = kf.smooth(data_full)

# Calculate anomaly score (residual)
anomaly_scores_kf = np.abs(data_full - smoothed_state_means.flatten())

# Store in a dataframe
kf_results = pd.DataFrame({
    'actual': data_full,
    'smoothed': smoothed_state_means.flatten(),
    'anomaly_score': anomaly_scores_kf
}, index=ts.index)

print("Kalman Filter smoothing complete.")
display(kf_results.head())

In [ ]:
# Plot 1: Actual vs. Smoothed
plt.figure(figsize=(12, 5))
plt.plot(kf_results['actual'], label='Actual Value (Noisy)', alpha=0.7)
plt.plot(kf_results['smoothed'], label='Kalman Filter Estimate', linestyle='--', color='green')
plt.title('Kalman Filter: Noisy vs. Smoothed Estimate')
plt.legend()
plt.show()

# Plot 2: Anomaly Scores
plt.figure(figsize=(12, 5))
plt.plot(kf_results['anomaly_score'], label='Kalman Filter Anomaly Score', color='green')
plt.title('Kalman Filter Anomaly Scores (Residuals)')
plt.xlabel('Time in Cycles')
plt.ylabel('Anomaly Score')
plt.legend()
plt.show()

In [ ]:
# Create a final results dataframe
df_final_scores = pd.DataFrame({
    'ARIMA_Score': arima_results['anomaly_score'],
    'KF_Score': kf_results['anomaly_score']
})

# We only have ARIMA scores for the 'test' portion. Let's compare there.
df_final_scores = df_final_scores.loc[split_point:]

# Scale the scores from 0 to 1 for comparison
scaler_scores = MinMaxScaler()
df_final_scores_scaled = pd.DataFrame(
    scaler_scores.fit_transform(df_final_scores),
    columns=df_final_scores.columns,
    index=df_final_scores.index
)

# Plot the comparison
plt.figure(figsize=(14, 7))
plt.plot(df_final_scores_scaled['ARIMA_Score'], label='ARIMA Anomaly Score (Scaled)')
plt.plot(df_final_scores_scaled['KF_Score'], label='Kalman Filter Anomaly Score (Scaled)')
plt.title('Baseline Model Anomaly Score Comparison (FD001, Unit 1, Sensor 7)')
plt.xlabel('Time in Cycles')
plt.ylabel('Normalized Anomaly Score')
plt.legend()
plt.show()